In [1]:
import numpy as np
import itertools
import picos as pic
import Fun as f
import mosek
import sys

import collections

In [11]:
def SDP(n,K,d,D):
    
    ##### SDP #####

    pic.available_solvers()
    
    Ap = pic.RealVariable("Ap", n+1)  # A column vector with n+1 elements.
    
    sdp = pic.Problem()
    
    #sdp.options["abs_*_tol"] = 1e-08
    
    #sdp.options["mosek_params"]={("MSK_DPAR_BASIS_TOL_X", 1e-09)}

    #sdp.options["mosek_params"]={("MSK_IPAR_INTPNT_SOLVE_FORM","MSK_SOLVE_PRIMAL")}
    
    #sdp.options["mosek_params"]={("MSK_IPAR_SIM_SOLVE_FORM","MSK_SOLVE_PRIMAL")}
    
    #sdp.options["mosek_params"]={("MSK_IPAR_PRESOLVE_USE","MSK_PRESOLVE_MODE_OFF")}
    
    #sdp.options["mosek_params"]={("MSK_IPAR_NUM_THREADS",200)}

    
    sdp.set_objective('find') 
       
    ##### Adding matrix constraints #####
    
    sdp.add_constraint(Ap[0]==1)
    
    ##### NPA #####
     
    ### Constructing Beta matrices ###
    
    Bmatrix=f.BetaMatrices(Ap,n) #### B_j(x)>>0 ####
    
    N=[]
    
    for k in range(int(n/2)+1):
        suma=0
        for t in range(n):
            for i in range(k,n-k+1):
                
                l=2*(i-t)
                
                b=f.Beta(i,i,k,t,n)
                
                if l>n or t>i:
                    
                    suma+=0
                    
                elif b>=0:
                    suma+=b/f.comb(n-2*k,i-k)
                #else:
                #   suma+=b*D**(-i)/(f.comb(n-2*k,i-k))
        N.append(suma)

    sdp.add_list_of_constraints(pic.block(Bmatrix[k])/N[k]>>0 for k in range(int(n/2)+1))
    
    
    ##### Adding K-L constrains #####
    
    ### K*B'_j-A'_j=0  for d > j >= 0 ###
    
    sdp.add_list_of_constraints((K*Ap[n-i]-Ap[i])== 0 for i in range(d)) 
    
    ### A_j>=0 ###
        
    for m in range(n+1):
        A=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*Ap[j] for j in range(m+1))
        sdp.add_constraint(A >= 0)

    ### K*B_j-A_j>=0 ###
    
    for m in range(n+1):   
                                    
        KBA=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*(K*Ap[n-j]-Ap[j]) for j in range(m+1))
        sdp.add_constraint(KBA >= 0)
    
    ##### Adding Shadow constrains #####
    
    ### sum(K_j*A'_j)>=0 ###
    
    for j in range(n+1):
        
        S=sum(f.K_fun(n-j,i,n)*Ap[i] for i in range(n+1))
        #sdp.add_constraint(S >= 0)
     
    ### Type II qubit codes  ###
    
    suma=0
        
    for m in range(0,n+1):
        
        if (m % 2) == 0:
            
            A=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*Ap[j] for j in range(m+1))
        
            suma += A
        
    #sdp.add_constraint(suma == 2**(n-int(round(np.log2(K)))))

    
    ### Convention for K=1 codes ###
    
    if K==1:
        
        sdp.add_list_of_constraints(Ap[i]==f.comb(n,i)/D**i for i in range(1,d))
    

    sol = sdp.solve(primals=False,solver='mosek')

    """
    print(Ap)
    
    a=[]
    for m in range(n+1):
        A=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*Ap[j] for j in range(m+1))
        a.append(np.round(A,2))
        #a.append(np.round(A,8))
    
    print(a)
    """
    if sol.problemStatus == "feasible":
        
        task=sdp.strategy.solver.int
        task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        print(task.solutionsummary(mosek.streamtype.msg))     
        
        #print(task.getsolution(mosek.soltype.itr)[3])
        #print(task.getsolution(mosek.soltype.itr)[4])
        #print(task.getprimalsolutionnorms(mosek.soltype.itr))
        return True    
    
    elif sol.problemStatus == "infeasible":
        task=sdp.strategy.solver.int
        task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        print(task.solutionsummary(mosek.streamtype.msg))
        
        return False
    else:
        task=sdp.strategy.solver.int
        task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        print(task.solutionsummary(mosek.streamtype.msg))
        
        return sol.problemStatus 


D=2


Codes=[[24,2**0,10]]

#Codes=[[35,1,1]]

#Codes=[[2,1,1]]

#Codes=[[26,1,10]]


for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

 


Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 0.0000000000e+00    nrm: 6e+06    Viol.  con: 6e-01    var: 0e+00    barvar: 0e+00  
  Dual.    obj: 3.0412837626e-07    nrm: 2e+04    Viol.  con: 2e-18    var: 5e-07    barvar: 8e-11  
None
[24, 1, 10]=True
